In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import ast
np.seterr(divide='ignore', invalid='ignore')
%matplotlib qt5

In [3]:
data = pd.read_csv('forks.csv')

In [4]:
areas = data['area']
lengths = data['length'] 
bin_scale=20
longs = data['normalized_internal_x']
counts = data['normalization_counts']

In [5]:
from rtseg.cells.plotting import generate_fork_plot

In [6]:
heatmap, mean_cell_lengths, (x, y) = generate_fork_plot(areas, lengths, longs, counts)

plt.imshow(heatmap, aspect='auto', interpolation='none', extent=[x[0], x[-1], y[-1], y[0]], origin='upper')
plt.plot(-0.5 * mean_cell_lengths, y, 'w', linewidth=2)
plt.plot(0.5 * mean_cell_lengths, y, 'w', linewidth=2)
plt.xlabel('Cell long axis (µm)')
plt.ylabel('Cell size (µm^2)')

Text(0, 0.5, 'Cell size (µm^2)')

In [7]:
mean_cell_lengths

array([ 1.54998378,  1.59395179,  1.66887195,         nan,  1.73079313,
        1.73636468,  1.83111858,  1.97497763,  2.04635954,  1.88393574,
        1.92966875,  1.88215439,  2.16219484,  2.04507083,  2.10881828,
        2.07634639,  2.29229938,  2.26061842,  2.40273692,  2.41490314,
        2.44421217,  2.43128967,  2.50391578,  2.566323  ,  2.659168  ,
        2.58120164,  2.63233816,  2.56089477,  2.72973262,  2.7311826 ,
        2.70035949,  3.012912  ,  3.06905901,  3.15087919,  3.10846871,
        3.1086096 ,  3.18162191,  3.26080274,  3.23269927,  3.26573434,
        3.25697045,  3.23628256,  3.21967627,  3.43281816,  3.54052596,
        3.36855862,  3.57503697,  3.73315912,  3.63344419,  3.83413482,
        3.41404968,  3.90598491,  4.34526937,  4.03167836,  3.99156377,
        4.00085595,  4.06477062,  4.09994337,         nan,  4.16164106,
        4.33808926,  4.43597175,  3.92590616,  4.39734251,  4.42214676,
        4.09730764,  4.98733949,  4.56046602,  4.156583  ,      

In [5]:
amin = np.quantile(areas, 0.005)
amax = np.quantile(areas, 0.98)

In [6]:
ainds = np.where(np.logical_and(areas >= amin, areas <= amax))[0]

In [7]:
longs = longs - 0.5

In [8]:
rescaled_lengths = lengths * longs
rescaled_lengths_filtered = rescaled_lengths[ainds]

In [9]:
rescaled_lengths.shape, rescaled_lengths_filtered.shape

((1025,), (1000,))

In [10]:
lmax = max(rescaled_lengths_filtered)
lmin = min(rescaled_lengths_filtered)
lminmax = max(lmax, -lmin)
lnrbins = round(2 * lminmax * bin_scale)
if lnrbins % 2 != 0:
    lnrbins += 1

In [11]:
lmax, lmin

(7.3832498726986175, -7.53183329711531)

In [12]:
lbins = np.linspace(-lminmax, lminmax, lnrbins)
anrbins = lnrbins
abins = np.linspace(amin, amax, anrbins)

In [13]:
len(lbins)

302

In [14]:
len(abins)

302

In [15]:
anrbins

302

In [16]:
heatmap = np.zeros(shape=(anrbins-1, lnrbins))
mean_cell_lengths = np.zeros((1, anrbins-1))

In [18]:
lbins.shape

(302,)

In [19]:
len(abins), anrbins

(302, 302)

In [30]:
heatmap = np.zeros(shape=(anrbins-1, lnrbins-1))
mean_cell_lengths = np.zeros((anrbins-1,))
for i in range(0, anrbins-1):
    sel_dots = np.logical_and(areas > abins[i], areas <= abins[i+1])
    sel_lengths = lengths[sel_dots]
    sel_counts = counts[sel_dots]
    sel_long = longs[sel_dots]
    h1,_ = np.histogram(sel_long * sel_lengths, lbins)
    normFactor = np.sum(1./sel_counts)
    heatmap[i, :] = h1 / normFactor
    mean_cell_lengths[i] = np.mean(sel_lengths)

# threshold heatmap 
heatmap[np.isnan(heatmap)] = 0
#

#heatmap[np.isnan(heatmap)] = 0

In [31]:
np.quantile(heatmap, 0.99)

0.0

In [21]:
mean_cell_lengths

array([ 1.54998378,  1.59395179,  1.66887195,         nan,  1.73079313,
        1.73636468,  1.83111858,  1.97497763,  2.04635954,  1.88393574,
        1.92966875,  1.88215439,  2.16219484,  2.04507083,  2.10881828,
        2.07634639,  2.29229938,  2.26061842,  2.40273692,  2.41490314,
        2.44421217,  2.43128967,  2.50391578,  2.566323  ,  2.659168  ,
        2.58120164,  2.63233816,  2.56089477,  2.72973262,  2.7311826 ,
        2.70035949,  3.012912  ,  3.06905901,  3.15087919,  3.10846871,
        3.1086096 ,  3.18162191,  3.26080274,  3.23269927,  3.26573434,
        3.25697045,  3.23628256,  3.21967627,  3.43281816,  3.54052596,
        3.36855862,  3.57503697,  3.73315912,  3.63344419,  3.83413482,
        3.41404968,  3.90598491,  4.34526937,  4.03167836,  3.99156377,
        4.00085595,  4.06477062,  4.09994337,         nan,  4.16164106,
        4.33808926,  4.43597175,  3.92590616,  4.39734251,  4.42214676,
        4.09730764,  4.98733949,  4.56046602,  4.156583  ,      

In [22]:
heatmap[np.isnan(heatmap)] = 0

In [23]:
abins.shape

(302,)

In [24]:
y = (abins[0:-1] + abins[1:])/2
x = lbins

In [26]:
plt.matshow(heatmap, aspect='auto', interpolation='none', extent=[x[0], x[-1], y[-1], y[0]], origin='upper')
plt.plot(-0.5 * mean_cell_lengths, y, 'w', linewidth=2)
plt.plot(0.5 * mean_cell_lengths, y, 'w', linewidth=2)
plt.xlabel('Cell long axis (µm)')
plt.ylabel('Cell size (µm^2)')

Text(0, 0.5, 'Cell size (µm^2)')

In [20]:
np.sum(heatmap)

890.0

In [45]:
import numpy as np
import matplotlib.pyplot as plt

# Create sample data
x = np.linspace(-100, -10, 10)
y = np.array([-8, -3])
data = np.random.randn(len(y), len(x))

# Display the image
plt.matshow(data, aspect='auto', interpolation='none', 
           extent=[x[0], x[-1], y[0], y[-1]], origin='lower')
plt.colorbar()
plt.show()


In [44]:
data.shape

(2, 10)

In [41]:
x[0]

-100.0

In [42]:
x[-1]

-10.0

In [115]:
heatmap.shape

(6549, 6550)

In [107]:
sel_long * sel_lengths

Series([], dtype: float64)

In [105]:
h1.shape

(6549,)

In [96]:
1./sel_counts

477    0.083333
478    0.083333
479    0.083333
480    0.083333
481    0.083333
482    0.083333
483    0.083333
484    0.083333
485    0.083333
486    0.083333
487    0.083333
488    0.083333
Name: normalization_counts, dtype: float64

In [75]:
np.sum(np.logical_and(areas > abins[0], areas <= abins[1]))

0

In [73]:
np.l

0       368.0
1       368.0
2       596.0
3       596.0
4       596.0
        ...  
1020    582.0
1021    371.0
1022    371.0
1023    455.0
1024    455.0
Name: area, Length: 1025, dtype: float64

### Parameters

In [3]:
bin_scale = 20
heatmap_threshold = 0.99